# Pothole Detection Dashboard

This notebook runs pothole detection using a **YOLOv12-style Ultralytics model** inside Jupyter.

**You’ll do three things here:**
1. Load your trained weights (`best.pt`)
2. Choose a source (video file or webcam)
3. Run detection and view results in a simple dashboard (video + statistics)

**Expected files (recommended):**
- `best.pt` — your trained model weights
- `demo.mp4` — sample test video


## 1) Install & Import Dependencies

This cell imports the libraries used by the notebook:
- **Ultralytics** for YOLO inference
- **OpenCV** for reading video/webcam frames
- **supervision** for converting detections + drawing boxes/labels
- **ipywidgets** for the notebook dashboard UI

If you already have these installed, you can just run the imports.


In [1]:
# If you're running locally, uncomment the next line:
# !pip -q install ultralytics supervision opencv-python ipywidgets matplotlib

import os
import time
import cv2
import numpy as np

from ultralytics import YOLO
import supervision as sv

from IPython.display import display, HTML
import ipywidgets as widgets

print("Imports loaded.")

FlashAttention is not available on this device. Using scaled_dot_product_attention instead.
Imports loaded.


## 2) Dashboard Styling

This cell injects CSS for the dashboard UI.


In [2]:
DASHBOARD_CSS = r'''
<style>

/* Global styles and variables */
.cv-scope{
    /* Root Variables */
    --accent: #7b1e1e;
    --accent2: #5c1313;
    --accent-rgb: 123, 30, 30;
    --bg: #fff7f7;
    --card: #ffffff;
    --text: #2b2b2b;
    --muted: #6b6b6b;
    --shadow: rgba(0,0,0,0.10);

    /* Layout */
    --cv-main-h: 560px;

    /* Font Style */
    font-family: Arial, sans-serif;
    color: var(--text);
}

/* container */
.cv-scope.cv-shell{
    background: var(--bg);
    border: 1px solid rgba(var(--accent-rgb), 0.22);
    border-radius: 16px;
    padding: 14px;
}

/* header */
.cv-scope .cv-header{
    background: linear-gradient(90deg, var(--accent), var(--accent2));
    color: white;
    padding: 14px 16px;
    border-radius: 14px;
    box-shadow: 0 6px 14px var(--shadow);
    margin-bottom: 12px;
    text-align: center;
}
.cv-scope .cv-header h2{
    margin: 0;
    font-size: 20px;
    letter-spacing: 0.2px;
}

/* panel (analysis stats) */
.cv-scope .cv-panel{
    background: var(--card);
    padding: 14px;
    border-radius: 14px;
    border: 1px solid rgba(var(--accent-rgb), 0.28);
    box-shadow: 0 10px 18px var(--shadow);

    height: var(--cv-main-h);
    overflow: auto;
}
.cv-scope .cv-panel-title,
.cv-scope .cv-panel h3{
    margin: 0 0 10px 0;
    color: var(--accent);
    border-bottom: 2px solid rgba(var(--accent-rgb), 0.35);
    padding-bottom: 6px;
    font-size: 16px;
}

.cv-scope .cv-item{
    margin: 10px 0;
    padding: 10px 12px;
    background: #ffffff;
    border: 1px solid rgba(0,0,0,0.06);
    border-radius: 10px;
    font-size: 14px;
}

.cv-scope .widget-button{
    border-radius: 10px !important;
}

/* layout helpers */
.cv-scope .cv-main-row{
    display:flex;
    gap:16px;
    align-items: stretch;
    flex-wrap:wrap;
}

/* video column */
.cv-scope .cv-video-wrap{
    flex:2;
    min-width:360px;
    height: var(--cv-main-h);

    display:flex;
    flex-direction: column;
}
.cv-scope .cv-btnrow{
    gap:10px;
    margin-bottom:10px;
}

.cv-scope .cv-video-frame{
    flex: 1 1 auto;
    min-height: 0;
    display:flex;
}

/* The <img> element itself */
.cv-scope .cv-video-frame img{
    width: 100%;
    height: 100%;
    display: block;
    object-fit: contain;
    border-radius: 12px;
    border: 1px solid rgba(var(--accent-rgb), 0.35);
    box-shadow: 0 4px 10px var(--shadow);
    background: #fff;
}

.cv-scope .cv-output{
    background: #ffffff;
    border: 1px dashed rgba(var(--accent-rgb), 0.35);
    border-radius: 14px;
    padding: 10px;
}

</style>
'''
display(HTML(DASHBOARD_CSS))

## 3) Paths + controls

This cell creates the controls you will use before running detection:

- **Source**: Video file or webcam  
- **Model path**: where your `.pt` weights are  
- **Video path**: used only if Source = Video file  
- **Conf / IoU**: detection thresholds  
- **Save output**: optionally save the annotated video to disk

You can change these anytime and re-run the later cells.


In [ ]:
MODEL_PATH_DEFAULT = "best.pt"
VIDEO_PATH_DEFAULT = "demo.mp4"

# Controls
source_choice = widgets.Dropdown(
    options=[("Video file (mp4/avi/mov)", "video"), ("Webcam (device 0)", "webcam")],
    value="video",
    description="Source:",
    layout=widgets.Layout(width="320px")
)

model_path = widgets.Text(
    value=MODEL_PATH_DEFAULT,
    description="Model:",
    placeholder="e.g., best.pt",
    layout=widgets.Layout(width="520px")
)

video_path = widgets.Text(
    value=VIDEO_PATH_DEFAULT,
    description="Video:",
    placeholder="e.g., demo.mp4",
    layout=widgets.Layout(width="520px")
)

conf_slider = widgets.FloatSlider(
    value=0.7, min=0.1, max=0.95, step=0.05,
    description="Conf:",
    readout_format=".2f",
    layout=widgets.Layout(width="520px")
)

iou_slider = widgets.FloatSlider(
    value=0.5, min=0.1, max=0.95, step=0.05,
    description="IoU:",
    readout_format=".2f",
    layout=widgets.Layout(width="520px")
)

max_frames = widgets.IntText(
    value=0,
    description="Max frames:",
    tooltip="0 = no limit (process entire stream)",
    layout=widgets.Layout(width="320px")
)

save_output = widgets.Checkbox(
    value=False,
    description="Save annotated video",
    indent=False
)

save_path = widgets.Text(
    value="annotated_output.mp4",
    description="Save as:",
    layout=widgets.Layout(width="520px")
)

display_every = widgets.IntSlider(
    value=1, min=1, max=10, step=1,
    description="Display every:",
    tooltip="Higher = faster UI (skips some frame updates)",
    layout=widgets.Layout(width="520px")
)

resize_width = widgets.IntSlider(
    value=960, min=480, max=1400, step=20,
    description="Resize width:",
    layout=widgets.Layout(width="520px")
)

display(
    widgets.HBox([source_choice, max_frames, save_output]),
    widgets.HBox([model_path]),
    widgets.HBox([video_path]),
    widgets.HBox([conf_slider]),
    widgets.HBox([iou_slider]),
    widgets.HBox([save_path]),
    widgets.HBox([display_every]),
    widgets.HBox([resize_width]),
)

## 4) (Optional) Paste your YOLO class labels

If you provide your dataset’s class names, the dashboard labels become human‑readable:

Example output: `pothole 0.82`

How to use:
- Put your class names in `CLASS_LABELS` **in the exact order** used in training.


In [4]:

# Leave these as None if you want to use model.names automatically.
CLASS_LABELS = None
CLASS_LABELS_MAP = None

## 5) Load the model + create annotators

This cell:
- Validates that your model file exists
- Loads the YOLO model once (so inference is faster)
- Prepares annotators that draw:
  - bounding boxes
  - labels (class name + confidence)

Run this again if you change the model path.


In [5]:
def _require_file(path: str, label: str) -> str:
    if not path:
        raise FileNotFoundError(f"{label} path is empty.")
    if not os.path.exists(path):
        raise FileNotFoundError(f"{label} not found: {path}")
    return path

def load_pipeline(model_path_str: str):
    _require_file(model_path_str, "Model weights (.pt)")

    model = YOLO(model_path_str)

    box_annotator = sv.BoxAnnotator(
        thickness=2,
        color=sv.Color.from_hex("#7b1e1e")
    )
    label_annotator = sv.LabelAnnotator(
        text_scale=0.7,
        text_thickness=1,
        text_color=sv.Color.WHITE,
        text_padding=10
    )
    return model, box_annotator, label_annotator

def resolve_class_name(class_id: int, model) -> str:
    
    # Priority: dict override > list override > model.names > fallback
    if CLASS_LABELS_MAP is not None and class_id in CLASS_LABELS_MAP:
        return str(CLASS_LABELS_MAP[class_id])
    if CLASS_LABELS is not None and 0 <= class_id < len(CLASS_LABELS):
        return str(CLASS_LABELS[class_id])
    
    # Ultralytics usually provides model.names as dict or list
    names = getattr(model, "names", None)
    if isinstance(names, dict) and class_id in names:
        return str(names[class_id])
    if isinstance(names, list) and 0 <= class_id < len(names):
        return str(names[class_id])
    return f"class_{class_id}"

model, box_annotator, label_annotator = load_pipeline(model_path.value)
print("Model loaded:", model_path.value)

Model loaded: best.pt


## 6) Frame processing function (one frame in → annotated frame out)

This cell defines the core inference step used by the dashboard:
1. Run YOLO prediction on a single frame  
2. Convert results to `supervision.Detections`  
3. Create labels like `class confidence`  
4. Draw the boxes + labels on the frame  


In [6]:
def process_frame(frame_bgr, *, model, box_annotator, label_annotator, conf: float, iou: float):
    results = model.predict(frame_bgr, conf=conf, iou=iou, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    det_count = len(detections)

    labels = []
    if det_count > 0:
        class_ids = detections.class_id if detections.class_id is not None else [None] * det_count
        confs = detections.confidence if detections.confidence is not None else [None] * det_count

        for cid, c in zip(class_ids, confs):
            if cid is None:
                name = "object"
            else:
                name = resolve_class_name(int(cid), model)
            if c is None:
                labels.append(f"{name}")
            else:
                labels.append(f"{name} {float(c):.2f}")

    annotated = box_annotator.annotate(scene=frame_bgr.copy(), detections=detections)
    annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

    return annotated, det_count

## 7) Run loop + dashboard UI

This cell builds the dashboard and runs detection.

Layout (unchanged):
- Start/Stop buttons **above** the video
- Video on the **left**
- Analysis Statistics on the **right**
- Video + stats are forced to the **same height** (no panel longer than the other)


In [7]:
# --- Dashboard widgets (video inside dashboard + working Stop) ---
import threading

status_w = widgets.HTML("<div class='cv-item'><b>Status:</b> Ready</div>")
model_w  = widgets.HTML(f"<div class='cv-item'><b>Model:</b> {model_path.value}</div>")
fps_w    = widgets.HTML("<div class='cv-item'><b>FPS:</b> --</div>")
det_w    = widgets.HTML("<div class='cv-item'><b>Detections:</b> 0</div>")
conf_w   = widgets.HTML(f"<div class='cv-item'><b>Conf:</b> {conf_slider.value:.2f}</div>")
iou_w    = widgets.HTML(f"<div class='cv-item'><b>IoU:</b> {iou_slider.value:.2f}</div>")

panel_title = widgets.HTML("<h3 class='cv-panel-title'>Analysis Statistics</h3>")
stats_panel = widgets.VBox([panel_title, status_w, model_w, fps_w, det_w, conf_w, iou_w])
stats_panel.add_class('cv-panel')

# Buttons
run_btn = widgets.Button(description="Run", button_style="success", icon="play")
stop_btn = widgets.Button(description="Stop", button_style="danger", icon="stop")
stop_btn.disabled = True
btn_row = widgets.HBox([run_btn, stop_btn])
btn_row.add_class('cv-btnrow')

# Video display
video_img = widgets.Image(value=b"", format="jpeg")
video_img.layout = widgets.Layout(width="100%", height="100%")
video_img.add_class('cv-video-frame')
video_wrap = widgets.VBox([btn_row, video_img])
video_wrap.add_class('cv-video-wrap')

header_html = widgets.HTML(
    '''
    <div class="cv-header">
      <h2 style="margin:0;">Pothole Detection Dashboard</h2>
    </div>
    '''
)

main_row = widgets.HBox([video_wrap, stats_panel])
main_row.add_class('cv-main-row')

dashboard = widgets.VBox([header_html, main_row])
dashboard.add_class('cv-scope')
dashboard.add_class('cv-shell')

display(dashboard)

_stop_event = threading.Event()
_worker = {"thread": None}

def _resize_keep_aspect(frame, target_w: int):
    h, w = frame.shape[:2]
    if w == target_w:
        return frame
    scale = target_w / float(w)
    new_h = int(h * scale)
    return cv2.resize(frame, (target_w, new_h), interpolation=cv2.INTER_AREA)

def _open_capture():
    if source_choice.value == "webcam":
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            raise RuntimeError("Could not open webcam (device 0). Try a different device index or use a video file.")
        return cap
    else:
        vp = _require_file(video_path.value, "Video file")
        cap = cv2.VideoCapture(vp)
        if not cap.isOpened():
            raise RuntimeError(f"Could not open video: {vp}")
        return cap

def _create_writer(save_to: str, fps: float, frame_shape):
    h, w = frame_shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    return cv2.VideoWriter(save_to, fourcc, fps if fps and fps > 0 else 25.0, (w, h))

def _bgr_to_jpeg_bytes(frame_bgr, quality: int = 80) -> bytes:
    ok, buf = cv2.imencode(".jpg", frame_bgr, [int(cv2.IMWRITE_JPEG_QUALITY), int(quality)])
    return buf.tobytes() if ok else b""

def _set_running_ui(is_running: bool):
    run_btn.disabled = is_running
    stop_btn.disabled = not is_running

def _worker_loop():
    global model, box_annotator, label_annotator

    try:
        status_w.value = "<div class='cv-item'><b>Status:</b> Starting...</div>"
        model_w.value  = f"<div class='cv-item'><b>Model:</b> {model_path.value}</div>"

        # Reload model each run
        model, box_annotator, label_annotator = load_pipeline(model_path.value)

        cap = _open_capture()
        src_fps = cap.get(cv2.CAP_PROP_FPS)
        if src_fps is None or src_fps <= 1e-6:
            src_fps = 0.0

        writer = None

        # FPS measurement
        t0 = time.time()
        frames_seen = 0

        # Frame limit
        max_f = int(max_frames.value) if int(max_frames.value) > 0 else None
        show_every = max(1, int(display_every.value))

        idx = 0
        while cap.isOpened() and not _stop_event.is_set():
            ok, frame = cap.read()
            if not ok:
                break

            idx += 1
            if max_f is not None and idx > max_f:
                break

            frame = _resize_keep_aspect(frame, int(resize_width.value))

            annotated, det_count = process_frame(
                frame,
                model=model,
                box_annotator=box_annotator,
                label_annotator=label_annotator,
                conf=float(conf_slider.value),
                iou=float(iou_slider.value),
            )

            if save_output.value and writer is None:
                writer = _create_writer(save_path.value, src_fps, annotated.shape)
            if writer is not None:
                writer.write(annotated)

            # Update stats
            frames_seen += 1
            dt = time.time() - t0
            fps_now = frames_seen / dt if dt > 0 else 0.0

            status_w.value = "<div class='cv-item'><b>Status:</b> Running</div>"
            fps_w.value    = f"<div class='cv-item'><b>FPS:</b> {fps_now:.2f}</div>"
            det_w.value    = f"<div class='cv-item'><b>Detections:</b> {det_count}</div>"
            conf_w.value   = f"<div class='cv-item'><b>Conf:</b> {conf_slider.value:.2f}</div>"
            iou_w.value    = f"<div class='cv-item'><b>IoU:</b> {iou_slider.value:.2f}</div>"

            # Update video (every N frames for speed)
            if (idx % show_every) == 0:
                video_img.value = _bgr_to_jpeg_bytes(annotated, quality=80)

        # End state
        if _stop_event.is_set():
            status_w.value = "<div class='cv-item'><b>Status:</b> Stopped</div>"
        else:
            status_w.value = "<div class='cv-item'><b>Status:</b> Finished</div>"

    except Exception as e:
        status_w.value = "<div class='cv-item'><b>Status:</b> Error</div>"
        raise
    finally:
        try:
            cap.release()
        except Exception:
            pass
        try:
            if writer is not None:
                writer.release()
        except Exception:
            pass

        _set_running_ui(False)

def on_run_clicked(_):
    if _worker["thread"] is not None and _worker["thread"].is_alive():
        return

    _stop_event.clear()
    _set_running_ui(True)
    status_w.value = "<div class='cv-item'><b>Status:</b> Starting...</div>"

    t = threading.Thread(target=_worker_loop, daemon=True)
    _worker["thread"] = t
    t.start()

def on_stop_clicked(_):
    _stop_event.set()
    status_w.value = "<div class='cv-item'><b>Status:</b> Stopping...</div>"

run_btn.on_click(on_run_clicked)
stop_btn.on_click(on_stop_clicked)

print("Detection dashboard is ready. Click Run to start. Stop will interrupt safely.")


Detection dashboard is ready. Click Run to start. Stop will interrupt safely.


## 8) Troubleshooting / Common Fixes

Use this section if something doesn’t run:

- **Model not found** → check `best.pt` path  
- **Video not found** → check `demo.mp4` path (or switch to webcam)  
- **Webcam not opening** → try a different device index (0/1/2)  
- **Slow UI** → increase “Display every” to 3–5  
- **No detections** → lower confidence slightly (e.g., 0.45–0.60)
